In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse as sp
import scipy



Heisenberg model Hamiltonian:

$$
H = \sum_{i} \sum_{j>i} J e^{\frac{-|i-j|}{\xi}} \mathbf{S}_i \cdot \mathbf{S}_j
$$

In [14]:
L = 10

In [15]:
# spins are on a 1D lattice, pointing up or down (z-direction)
# only Sz operator relevant.
Sz = sp.csr_matrix([[1., 0.], [0., -1.]])

def sigma_z(L=10):
    return_list = []
    for i in range(L):
        # i * id, then S_z, then (L-i-1) * id
        operator = sp.kron(sp.eye(2**i), Sz, format='csr')
        operator = sp.kron(operator, sp.eye(2**(L-i-1)), format='csr')
        return_list.append(operator)

    return return_list


def gen_hamiltonian(sz_list, J=1.0, xi=0.1):
    L = len(sz_list)
    H = sp.csr_matrix((2**len(sz_list), 2**len(sz_list)), dtype=np.float64)
    for j in range(L):
        for i in range(j + 1, L):
            H += J * np.exp(-abs(i-j) / xi) * sz_list[i].dot(sz_list[j])

    return H

def compress(psi, L, chi_max):
    return_list = []
    psi = psi.reshape((1, 2**L)) # initial reshape
    chi_n = 1 # only for the first one
    for n in range(L):
        
        psi_new = psi.reshape((chi_n * 2, 2**(L-n-1))) # psi _ Ln _ Rn+1
        # print("psi_new shape:", psi_new.shape)

        #SVD decomposition
        U, S, Vh = np.linalg.svd(psi_new, full_matrices=False)

        keep = np.argsort(S)[:: -1][:chi_max]
        M_n = U[:,keep]
        lambda_ = S[keep]
        psitilde = Vh[keep,:]

        M_new = M_n.reshape((chi_n, 2, -1))  # (chi_n, 2, chi_n+1)

        return_list.append(M_new)

        chi_n = M_new.shape[2]  # new chi_n
        psi = lambda_[:, np.newaxis] * psitilde # prepare for next step
        
    return return_list

H = gen_hamiltonian(sigma_z(L), J=1.0, xi=1.0)

In [16]:
eig_values, gs = scipy.sparse.linalg.eigsh(H, k=3, which='SA') 
ground_state = np.real(gs[:, 0])

psi = ground_state
res = compress(psi, L, chi_max=1000)


In [18]:
for n, M in enumerate(res):
    print(M.shape)

(1, 2, 2)
(2, 2, 4)
(4, 2, 8)
(8, 2, 16)
(16, 2, 32)
(32, 2, 16)
(16, 2, 8)
(8, 2, 4)
(4, 2, 2)
(2, 2, 1)
